(1_Ocho_Escuelas)=
# <span style="color:#F72585"><center>Ejemplo 1: Ocho Escuelas</center></span>


## <span style="color:#4361EE">Introducción</span>


Se introduce en este cuaderno un ejemplo clásico utilizado en la enseñanza de la Estadística Bayesiana. En particular el ejemplo ha sido usado por {cite}`gelman2003bayesian` y aparece en los manuales de usuario de Stan y TensorFlow Probability.

## <span style="color:#4361EE">Un primer modelo general</span>


### <span style="color:#4CC9F0">Modelo Estadístico</span>


Supongamos que tenemos observaciones $[y_n |x_n], n=1,\ldots,N$, y asumamos el modelo Bayesiano

$$
\begin{align}
y_n &\sim \mathcal{N}(\alpha + \beta x_n, \sigma^2),\quad n=1,\ldots,N\\
\alpha &\sim \mathcal{N}(0, 100)\\
\beta &\sim \mathcal{N}(0, 100)\\
\sigma &\sim \mathcal{C}auchy(0, 25) 1_{\sigma>0}\\
\end{align}
$$


¿Cuáles serían las distribuciones a priori?

## <span style="color:#4361EE">Primera Implementación</span>


Entonces, el modelo  se escribe de la siguiente manera:

````{tab} Pyro
```python
def model(x, y=None):
    alpha = pyro.sample("alpha",dist.Normal(0,10))
    beta = pyro.sample("beta",dist.Normal(0,10))
    sigma = pyro.sample("sigma",dist.Cauchy(0,25),constraint=constraints.positive)
    mu = alpha + beta * x
    with pyro.plate("plate"):
        pyro.sample("y",dist.Normal(alpha+beta*x,sigma),obs=y )
```
````
````{tab} TFP
```python
def model(x):
    return tfd.JointDistributionSequential([
        tfd.Sample(tfd.Normal(loc=0., scale=10., name="alpha"), sample_shape=1), #alpha
        tfd.Sample(tfd.Normal(loc=0., scale=10., name='beta'), sample_shape=1), #beta
        tfd.Sample(tfd.HalfCauchy(loc=0., scale=25., name='sigma'), sample_shape=1), #sigma  
        lambda alpha,beta,sigma,x: (
            tfd.Normal(loc= alpha+beta*x, scale=sigma, name='y')
        )  
    ])
```
````
````{tab} Stan
``` r
data { 
int<lower=0> N;
vector[N] y;
vector[N] x;
}


parameters {
real alpha;
real beta;
real<lower=0> sigma;
}


model {
alpha ~ normal(0,10);
beta ~ normal(0,10);
sigma ~ cauchy(0,5);
for (n in 1:N)
   y[n] ~ normal(alpha + beta * x[n], sigma);
}
```
````

```{admonition} Cuidado
:class: caution
Recuerde que en TFP, Pyro y Stan el segundo parámetro de la distribución normal es la desviación estándar y no la varianza.
```


## <span style="color:#4361EE">Ejemplo 1. Ocho escuelas</span>


El ejemplo de "ocho escuelas" aparece en la Sección 5.5 de {cite}`gelman2003bayesian`, en donde se estudian los efectos del entrenamiento de ocho escuelas.

1. Los datos se refieren al estudio del Educational Testing Service para analizar el efecto del entrenamiento.

2. Los datos provienen del examen  SAT-V en ocho escuelas secundarias

3.  No hay razón previa para creer que algún programa de entrenamiento fue:

- Más efectivo que los demás

- Más similar a otros




### <span style="color:#4CC9F0">Los datos</span>


Los datos corresponden a la estimación (estandarizada) de un puntaje realizada en las ocho escuelas observadas.


```{table}
|School |Estim. Treatment Effect | Estim. Stand. Error |
|---|---|---|
|A |28| 15|
|B |8 |10|
|C| -3| 16|
|D| 7| 11|
|E| -1 |9|
|F| 1| 11|
|G |18| 10|
|H| 12| 18|
```

````{tab} Pyro
```python
# Los datos
J = 8 #Número de escuelas
y = torch.tensor([28, 8, -3, 7, -1, 1, 18, 12]).type(torch.Tensor) #Estimaciones de los efectos del tratamiento
sigma = torch.tensor([15, 10, 16, 11, 9, 11, 10, 18]).type(torch.Tensor) #Estimacion de la desviacion de los efectos del tratamiento
schools = np.array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
                    'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'])

schools_dat = {'J': J,
               'y': y,
               'sigma': sigma}
schools_dat 
```
````

````{tab} TFP
```python
# Los datos
J = 8 #Número de escuelas
y = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.], dtype=np.float32) #Estimaciones de los efectos del tratamiento
sigma = np.array([15., 10., 16., 11.,  9., 11., 10., 18.], dtype=np.float32) #Estimacion de la desviacion de los efectos del tratamiento
schools = np.array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
                    'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'])

schools_dat = {'J': J,
               'y': y,
               'sigma': sigma}
schools_dat 
```
````

````{tab} Stan
``` r
schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}
schools_dat 
```
````

## <span style="color:#4361EE">Primer modelo</span>

Eight Schools: No Pooling

- Cada escuela es tratada individualmente. A prioris impropias. Cada media de las escuelas proviene de una distribución diferente.

$$
\begin{equation}
\begin{split}
y_i &\sim \mathcal{N}(\theta_i,\sigma_i^2), \text{ known } \sigma_i^2\\
\theta_i &\propto 1
\end{split}
\end{equation}
$$

````{tab} Pyro
```python
def schools_code_01(J, y, sigma):
    theta= pyro.param('theta', torch.ones(J))
    with pyro.plate("plate"):
        pyro.sample("y",dist.Normal(theta,sigma),obs=y )
```
````

````{tab} TFP
```python
def schools_code_01(y,sigma):
    return tfd.JointDistributionSequential([ 
        mu= y
        lambda mu,sigma: (
            tfd.Normal(loc= mu, scale=sigma, name='y')
        )  
    ])

```
````

````{tab} Stan
```r
schools_code_01 ="""

data {
int<lower=0> J; // # schools
real y[J]; // estimated treatment
real<lower=0> sigma[J]; // std err of effect
}

parameters {
real theta[J]; // school effect
}

model {
y ~ normal(theta, sigma);
}
"""

```
````

## <span style="color:#4361EE">Segundo Modelo</span>


Eight Schools: Complete Poolin. Todas la medias provienen de una única distribución.

- Todas las escuelas tomadas conjuntamente.

$$
y_i \sim \mathcal{N}(\theta,\sigma_i^2), \text{ known } \sigma_i^2
$$

````{tab} Pyro
```python
def schools_code_02(J, y,sigma):
    theta=pyro.param('theta',torch.ones(J))
    return pyro.sample("obs", dist.Normal(theta,sigma),obs=y)
```
````

````{tab} TFP
```python
def schools_code_02(J, y, sigma):
    theta = y
    return tfd.JointDistributionSequential([ 
        lambda theta,sigma: (
            tfd.Normal(loc= theta, scale=sigma, name='y')
        )  
    ])
```
````

````{tab} Stan
```r
schools_code_02 ="""

data {
int<lower=0> J; // # schools
real y[J]; // estimated treatment
real<lower=0> sigma[J]; // std err of effect
}

parameters {
real theta; // pooled school effect
}

model {
y ~ normal(theta, sigma);
}
"""
# compile the  model
sm_02 = pystan.StanModel(model_code=schools_code_02)
# extract the samples
fit_02 = sm_02.sampling(data=schools_dat, iter=1000, chains=4)
```
````

## <span style="color:#4361EE">Tercer Modelo</span>


Eight Schools: Partial Pooling. Estimamos una media global y ahora cada media de las escuelas es una muestra de otra distribución  global.
    
- Estima una media global $\mu$,la cual es un hiperparámetro  para el modelo y fijamos  otro hiperparámetro $\tau = 25$ para esa distribución.


$$
\begin{align}
y_i &\sim \mathcal{N}(\theta_i,\sigma_i^2), \text{ known } \sigma_i^2\\
\theta_i &\sim \mathcal{N}(\mu, \tau^2) , \text{ known } \tau^2
\end{align}
$$

Así hemos supuesto que hay una media global $\mu$ de tal manera que las medias $\theta_i$ de las escuelas son generadas a partir de una distribución normal con media $\mu$.

````{tab} Pyro
```python
# Los datos
schools_dat_03 = {'J': 8,
               'y': torch.tensor([28, 8, -3, 7, -1, 1, 18, 12]).type(torch.Tensor),
               'sigma': torch.tensor([15, 10, 16, 11, 9, 11, 10, 18]).type(torch.Tensor),
               'tau': 25}
```
````

````{tab} TFP
```python
schools_dat_03 = {'J': 8,
               'y': np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.], dtype=np.float32),
               'sigma': np.array([15., 10., 16., 11.,  9., 11., 10., 18.], dtype=np.float32),
               'tau': 25}
```
````

````{tab} Stan
```r
# Los datos
schools_dat_03 = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18],
               'tau': 25}
```
````

Una vez que agregamos $\tau$ a nuestro conjunto de datos podemos formular el modelo

````{tab} Pyro
```python
def schools_code_03(J,y,sigma,tau):
    mu = pyro.param('mu',torch.ones(J))
    theta = pyro.sample('theta',dist.Normal(mu,tau))
    with pyro.plate("plate"):
        pyro.sample("y",dist.Normal(theta,sigma),obs=y )   
```
````
````{tab} TFP
```python
def schools_code_03(J, y, sigma,tau):
    return tfd.JointDistributionSequential([ 
        #mu
        tfd.Uniform(low=tf.cast(0,dtype),high=tf.cast(1,dtype))
        #thetas
        lambda mu, tau: tfd.Sample(tfd.Normal(loc=mu,scale=tau,name='theta'),sample_shape=n)
        # y
        lambda thetas, sigma: tfd.independent(tfd.normal (loc= thetas, scale= sigma,name='y'))
        )  
    ])
```
````
````{tab} Stan
```r
schools_code_03 ="""

data {
int<lower=0> J; // # schools
real y[J]; // estimated treatment
real<lower=0> sigma[J]; // std err of effect
real<lower=0> tau; // variance between schools
}

parameters {
real theta[J]; // school effect
real mu; // mean for schools
}

model {
theta ~ normal(mu, tau);
y ~ normal(theta, sigma);
}
"""
# compile the  model
sm_03 = pystan.StanModel(model_code=schools_code_03)
# extract the samples
fit_03 = sm_03.sampling(data=schools_dat_03, iter=1000, chains=4)
```
````


## <span style="color:#4361EE">Cuarto Modelo</span>


Eight Schools: modelo jerárquico completo
    
• Estima los  hiperparametros $\mu$ and $\tau$




$$
\begin{align}
y_i &\sim \mathcal{N}(\theta_i,\sigma_i^2), \text{ known } \sigma_i^2\\
\theta_i &\sim \mathcal{N}(\mu, \tau)
\end{align}
$$

````{tab} Pyro
```python
def schools_code_04(J,y,sigma):
    tau = pyro.param('tau',torch.ones(J), constraint = positive)
    mu = pyro.param('mu',torch.ones(J))
    theta = pyro.sample('theta',dist.Normal(mu,tau))
    with pyro.plate("plate"):
        pyro.sample("y",dist.Normal(theta,sigma),obs=y )   
```
````
````{tab} TFP
```python
def schools_code_04(J, y, sigma):
    return tfd.JointDistributionSequential([ 
        #mu
        tfd.Uniform(low=tf.cast(0,dtype),high=tf.cast(1,dtype))
        #tau
        tfd.HalfCauchy(loc=tf.cast(0,dtype),scale=tf.cast(5,dtype))
        #thetas
        lambda mu, tau: tfd.Sample(tfd.Normal(loc=mu,scale=tau,name='theta'),sample_shape=n)
        # y
        lambda thetas, sigma: tfd.independent(tfd.normal (loc= thetas, scale= sigma,name='y'))
        )  
    ])
```
````
````{tab} Stan
```r
schools_code_04 ="""

data {
int<lower=0> J; // # schools
real y[J]; // estimated treatment
real<lower=0> sigma[J]; // std err of effect
}

parameters {
real theta[J]; // school effect
real mu; // mean for schools
real<lower=0> tau; // variance between schools
}

model {
theta ~ normal(mu, tau);
y ~ normal(theta, sigma);
}
"""
# compilar the  model
sm_04 = pystan.StanModel(model_code=schools_code_04)
# extract the samples
fit_04 = sm_04.sampling(data=schools_dat, iter=1000, chains=4)
```
````


## <span style="color:#4361EE">Quinto Modelo</span>



Modelo con efectos aleatorios
    
• Estima los hiperparámetros $\mu$ and $\tau$

• Predice los efectos aleatorios $\eta_i$


$$
\begin{align}
y_i &\sim \mathcal{N}(\theta_i,\sigma_i^2), \text{ known } \sigma_i^2\\
\theta_i & = \mu + \tau \times \eta_i \\
\eta_i &\sim \mathcal{N}(0, 1)
\end{align}
$$

````{tab} Pyro
```python
def schools_code_05(J,y,sigma):
    tau = pyro.param('tau',torch.ones(J), constraint = constraints.positive)
    mu = pyro.param('mu',torch.ones(J))
    eta = pyro.sample('eta',dist.Normal(0,1))
    theta = mu + tau*eta
    with pyro.plate("plate"):
        pyro.sample("y",dist.Normal(theta,sigma),obs=y)   
```
````
````{tab} TFP
```python
def schools_code_05(J, y, sigma):
    return tfd.JointDistributionSequential([ 
        #mu
        tfd.Uniform(low=tf.cast(0,dtype),high=tf.cast(1,dtype))
        #tau
        tfd.HalfCauchy(loc=tf.cast(0,dtype),scale=tf.cast(5,dtype))
        #etas
        tfd.Sample(tfd.Normal(loc=torch.zeros(J),scale=torch.ones(J)))
        #thetas
        lambda mu, tau, etas: mu+tau*etas
        # y
        lambda thetas, sigma: tfd.independent(tfd.normal (loc= thetas, scale= sigma,name='y'))
        )  
    ])
```
````

````{tab} Stan
```r
schools_code_05 = """

data {
    int<lower=0> J; // number of schools
    vector[J] y; // estimated treatment effects
    vector<lower=0>[J] sigma; // s.e. of effect estimates
}

parameters {
    real mu;
    real<lower=0> tau;
    vector[J] eta;
}

transformed parameters {
    vector[J] theta;
    theta = mu + tau * eta;
}
model {
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}
"""
# compile the  model
sm_05 = pystan.StanModel(model_code=schools_code_05)
# extract the samples
fit_05 = sm_05.sampling(data=schools_dat, iter=1000, chains=4)
```
````

## <span style="color:#4361EE">Tarea</span>


Reescriba la tarea de la sección [Interpretación de la Media Posterior](EAP) utilizando Stan y lo aprendido en este cuaderno.

## <span style="color:#4361EE">Referencias</span>


```{bibliography}
:filter: docname in docnames
```